In [1]:
print("all ok!")

all ok!


In [2]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain.agents import Tool
from langchain_community.utilities import GoogleSerperAPIWrapper

from typing import Annotated
from typing_extensions import TypedDict
from dotenv import load_dotenv
import gradio as gr
import requests
import os
import asyncio

In [3]:
load_dotenv()

True

In [4]:
# Defining a state

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder=StateGraph(State)

In [5]:
import os
import requests

PUSHOVER_URL="https://api.pushover.net/1/messages.json"
PUSHOVER_USER=os.getenv("PUSHOVER_USER")
PUSHOVER_TOKEN=os.getenv("PUSHOVER_TOKEN")

def push(text: str):
    """Send a push notification to the user"""
    requests.post(PUSHOVER_URL, data={"token":PUSHOVER_TOKEN, "user":PUSHOVER_USER, "message":text})

push_tool=Tool(
    name="send_push_notifications",
    func=push,
    description="Useful for when you need to send a push notification"
)

In [6]:
# nest_asyncio is used to run nested event loops
import nest_asyncio
nest_asyncio.apply()

In [7]:

memory = MemorySaver()

In [8]:
async def setup_browser_tools():
    """Setup browser tools with proper resource management"""
    serper=GoogleSerperAPIWrapper()
    search_tool=Tool(
        name="search",
        func=serper.run,
        description="Useful for when you need to more information from online search"
        )
    return [search_tool]


In [9]:

async def getting_graph(tools):
    # Use a more capable model
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)  # or "gpt-3.5-turbo" if gpt-4 is not available
    # Bind tools to LLM
    llm_with_tools = llm.bind_tools(tools)
    # Create agent function
    def agent(state: State):
        messages = state["messages"]
        print(f"🤖 Agent processing {len(messages)} messages")
        
        try:
            response = llm_with_tools.invoke(messages)
            print(f"🤖 Agent response type: {type(response)}")
            
            # Check if tool calls are present
            if hasattr(response, 'tool_calls') and response.tool_calls:
                print(f"🔧 Agent decided to use tools: {[tc['name'] for tc in response.tool_calls]}")
            else:
                print("💬 Agent responding with text only")
                
            return {"messages": [response]}
        except Exception as e:
            print(f"❌ Agent error: {e}")
            import traceback
            traceback.print_exc()
            # Return a simple error message
            from langchain_core.messages import AIMessage
            return {"messages": [AIMessage(content=f"I encountered an error: {str(e)}")]}
    
    # Build a new graph instance
    graph_builder = StateGraph(State)
    
    # Add nodes
    graph_builder.add_node("agent", agent)
    graph_builder.add_node("tools", ToolNode(tools))
    
    # Add edges
    graph_builder.add_edge(START, "agent")
    graph_builder.add_conditional_edges(
        "agent",
        tools_condition,
        "tools"
    )
    graph_builder.add_edge("tools", "agent")
    
    # Compile graph
    graph = graph_builder.compile(checkpointer=memory)
    
    return graph

In [10]:

async def main():
    try:
        # Setup browser tools
        search_tool = await setup_browser_tools()
        print("✅ Browser tools setup completed!")
        
        # Combine all tools
        all_tools = search_tool + [push_tool]
        print(f"✅ Combined {len(all_tools)} tools total")
        
        # Print available tools for debugging
        print("🛠️ Available tools:")
        for tool in all_tools:
            print(f"  - {tool.name}: {tool.description}")
        
        # Create the graph
        graph = await getting_graph(all_tools)
        print("✅ LangGraph created successfully!")

        # Now setup Gradio interface
        config = {"configurable": {"thread_id": "2"}}
        
        async def chat(message, history):
            print(f"👤 User input: {message}")
            
            try:
                # Use HumanMessage object instead of dictionary
                result = await graph.ainvoke(
                    {"messages": {"role":"user", "content":message}}, 
                    config=config
                )
                
                last_message = result["messages"][-1]
                response_content = last_message.content
                
                # Log the response
                print(f"🤖 Assistant response: {response_content}")
                
                return response_content
                
            except Exception as e:
                error_msg = f"Sorry, I encountered an error: {str(e)}"
                print(f"❌ Chat error: {e}")
                import traceback
                traceback.print_exc()
                return error_msg
        
        # Launch Gradio interface
        print("🚀 Launching Gradio interface...")

        demo = gr.ChatInterface(
            chat,
            title="AI Web Assistant",
            description="I can browse the web and search for information for you! Try asking me to navigate to a website or search for something.",
            type="messages"
        )
        
        demo.launch(share=False, inbrowser=True,debug=True)

    except Exception as e:
        print(f"❌ Error occurred: {e}")
        import traceback
        traceback.print_exc()
        
    finally:
        print("all done!!!")

In [11]:

if __name__ == "__main__":
    print("🚀 Running main application...")
    asyncio.run(main())

🚀 Running main application...
✅ Browser tools setup completed!
✅ Combined 2 tools total
🛠️ Available tools:
  - search: Useful for when you need to more information from online search
  - send_push_notifications: Useful for when you need to send a push notification
✅ LangGraph created successfully!
🚀 Launching Gradio interface...
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


👤 User input: hi there
🤖 Agent processing 1 messages
🤖 Agent response type: <class 'langchain_core.messages.ai.AIMessage'>
💬 Agent responding with text only
🤖 Assistant response: Hello! How can I assist you today?
👤 User input: what is current gdp of America
🤖 Agent processing 3 messages
🤖 Agent response type: <class 'langchain_core.messages.ai.AIMessage'>
🔧 Agent decided to use tools: ['search']
🤖 Agent processing 5 messages
🤖 Agent response type: <class 'langchain_core.messages.ai.AIMessage'>
💬 Agent responding with text only
🤖 Assistant response: The current GDP of the United States in 2023 is approximately 27.72 trillion USD.
👤 User input: Nice to meet you
🤖 Agent processing 7 messages
🤖 Agent response type: <class 'langchain_core.messages.ai.AIMessage'>
💬 Agent responding with text only
🤖 Assistant response: Nice to meet you too! If you have any more questions or need assistance with anything else, feel free to ask.
Keyboard interruption in main thread... closing server.
all done!